References
WESAD: https://archive.ics.uci.edu/dataset/465/wesad+wearable+stress+and+affect+detection
TabTransformer: https://aravindkolli.medium.com/mastering-tabular-data-with-tabtransformer-a-comprehensive-guide-119f6dbf5a79


Steps:
1. Import Dataset
2. Train-test split and Data Loader
3. Transformer/ Neural network
    1) Create a model
    2) Choose a loss function
    3) Set an optimizer 
    4) Run a training loop
        Calculate loss (Forward pass)
        Compute gradients (Backpropagation)
        Updating model parameters
4. Evaluation

In [36]:
#1. Import Dataset
import os
import pickle
import numpy as np
from scipy.signal import resample
from scipy.stats import mode
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [37]:
from torch.utils.data import random_split
import torch.nn as nn
import torch.optim as optim

In [38]:
class WESADDataset(Dataset):
    def __init__(self, data_path, window_size=128, overlap=0.0):
        self.data_path = data_path
        self.window_size = window_size
        self.overlap = overlap
        self.signal_names = ['ACC','Resp','EDA','Temp','ECG','EMG']  
        self.data, self.labels, self.subjects = self.load_dataset()
    
    def load_dataset(self):
        subjects = [f'S{i}' for i in range(1, 18) if i not in [1, 12]]  # S1 and S12 are not available (Problem with sensors)
        all_data = []
        all_labels = []
        all_subjects = []
        
        orig_fs = 700
        target_fs = 32
        
        for subject in subjects:
            subj_dir = os.path.join(self.data_path, subject)
            data_file = os.path.join(subj_dir, f'{subject}.pkl')
            
            if not os.path.exists(data_file):
                print(f'Warning: {data_file} does not exist')
                continue
            
            try:
                with open(data_file, 'rb') as f:
                    raw = torch.load(f) if self.data_path.endswith('.pt') else pickle.load(f, encoding='latin1')
                
                # Extract chest data and label
                chest_data = raw['signal']['chest']
                labels = raw['label']
                
                # Process signals
                signals = []
                for name in self.signal_names:
                    if name in chest_data:
                        sig = chest_data[name]
                        
                        # Handle multi-dimensional signals (like ACC with x,y,z components)
                        if len(sig.shape) > 1:
                            if name == 'ACC':
                                # For accelerometer, compute magnitude from 3D components
                                if sig.shape[1] == 3:  # x, y, z components
                                    sig = np.sqrt(np.sum(sig**2, axis=1))  # Magnitude
                                else:
                                    sig = sig.flatten()
                            else:
                                sig = sig.flatten()
                        
                        # Resample signal
                        sig_resampled = resample(sig, int(len(sig) * target_fs / orig_fs))
                        signals.append(sig_resampled)
                    else:
                        print(f'Warning: {name} missing for {subject}')
                
                if len(signals) != len(self.signal_names):
                    print(f'Skipping {subject} due to missing modalities')
                    continue
                
                # Ensure all signals have same length
                min_len = min(map(len, signals))
                signals = [s[:min_len] for s in signals]
                signal_matrix = np.stack(signals, axis=1)
                
                # Resample labels
                labels_resampled = resample(labels, min_len)
                labels_resampled = np.round(labels_resampled).astype(int)
                
                # Create sliding windows
                win_data, win_labels = self.create_windows(signal_matrix, labels_resampled)
                
                all_data.extend(win_data)
                all_labels.extend(win_labels)
                all_subjects.extend([subject]*len(win_data))
                
                print(f'Loaded {len(win_data)} sliding windows for {subject}')
                
            except Exception as e:
                print(f'Error processing {subject}: {e}')
                continue
        
        return np.array(all_data), np.array(all_labels), np.array(all_subjects)
    
    def create_windows(self, data, labels):
        step = int(self.window_size * (1 - self.overlap))
        windows = []
        window_labels = []
        
        for start in range(0, data.shape[0] - self.window_size + 1, step):
            end = start + self.window_size
            label_window = labels[start:end]
            
            # Handle newer scipy versions
            mode_result = mode(label_window, keepdims=True)
            lbl = int(mode_result[0][0])
            
            if lbl == 1:  # Baseline
                windows.append(data[start:end])
                window_labels.append(0)
            elif lbl == 2:  # Stress
                windows.append(data[start:end])
                window_labels.append(1)
        
        return windows, window_labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)


In [5]:
DATASET_PATH = '/Users/kumar/Library/Mobile Documents/com~apple~CloudDocs/Phoenix/OVGU/HiWi2/Tasks/10_WESAD/WESAD.nosync'

ds = WESADDataset(DATASET_PATH)

Loaded 440 sliding windows for S2
Loaded 445 sliding windows for S3
Loaded 449 sliding windows for S4
Loaded 460 sliding windows for S5
Loaded 458 sliding windows for S6
Loaded 457 sliding windows for S7
Loaded 460 sliding windows for S8
Loaded 456 sliding windows for S9
Loaded 476 sliding windows for S10
Loaded 465 sliding windows for S11
Loaded 461 sliding windows for S13
Loaded 464 sliding windows for S14
Loaded 464 sliding windows for S15
Loaded 463 sliding windows for S16
Loaded 476 sliding windows for S17


In [ ]:
# Window size: How many timesteps or consecutive records each sample contains
# Batch size: How many independent samples are processed in parallel

In [ ]:
print(len(ds)) # How 6894?, all devices sampled at same rate?
input_sample, label_sample = ds[0]
print(len(input_sample)) # Time steps = 128, (window size is downsampled from 700 Hz to 32Hz, 128/32 = 4 seconds of data per window)
print(len(input_sample[0])) # ['ACC','Resp','EDA','Temp','ECG','EMG'], 6 sensors
print('Input sample:', input_sample) # 128 * 6
print('Label sample:', label_sample) # 0 for Baseline, 1 for Stress label for 4 seconds window

6894
128
6
Input sample: tensor([[ 9.5370e-01,  2.2468e+00,  5.5277e+00,  2.9131e+01, -1.4182e-01,
         -6.0475e-03],
        [ 9.1147e-01,  2.3274e+00,  5.5262e+00,  2.9136e+01, -1.3497e-01,
          6.8507e-05],
        [ 9.0827e-01,  2.3982e+00,  5.5229e+00,  2.9145e+01, -9.1329e-02,
         -3.4008e-03],
        [ 9.2792e-01,  2.4003e+00,  5.5208e+00,  2.9142e+01, -1.2794e-01,
          5.1459e-05],
        [ 9.3718e-01,  2.4020e+00,  5.5201e+00,  2.9131e+01, -1.3628e-01,
         -4.2756e-03],
        [ 9.3415e-01,  2.3529e+00,  5.5167e+00,  2.9139e+01, -5.8765e-02,
         -2.8011e-03],
        [ 9.2376e-01,  2.2870e+00,  5.5171e+00,  2.9137e+01,  7.2924e-02,
         -3.5566e-03],
        [ 9.2641e-01,  2.2015e+00,  5.5147e+00,  2.9140e+01,  6.3188e-02,
         -2.0663e-03],
        [ 9.3307e-01,  2.0973e+00,  5.5099e+00,  2.9133e+01,  3.0525e-02,
         -2.8087e-03],
        [ 9.3486e-01,  1.9369e+00,  5.5108e+00,  2.9139e+01,  1.3204e-02,
         -2.8071e-03],
     

In [ ]:
# 2. Train test size

total_size = len(ds)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

print(total_size, train_size, test_size)

6894 5515 1379


In [14]:
# Train test split
train_dataset, test_dataset = random_split(ds, [train_size, test_size])

In [ ]:
# Data Loaders
train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle = True) # batch size = grouping 32 samples
test_dataloader = DataLoader(test_dataset, batch_size = 32, shuffle = True)

# Sample batch
for batch_inputs, batch_labels in train_dataloader:
    print('batch_inputs:', batch_inputs)
    print('batch_labels:', batch_labels)
    print(len(batch_inputs))
    print(len(batch_inputs[0]))
    break

batch_inputs: tensor([[[ 9.2007e-01, -2.8284e+00,  9.7674e-01,  3.3479e+01,  2.1780e-03,
           2.7065e-04],
         [ 9.2262e-01, -2.8378e+00,  9.7203e-01,  3.3481e+01,  2.3069e-01,
          -2.5320e-03],
         [ 9.1597e-01, -2.8122e+00,  9.7693e-01,  3.3481e+01,  3.4513e-02,
          -2.8421e-03],
         ...,
         [ 9.2315e-01, -2.0387e+00,  9.7202e-01,  3.3490e+01, -1.0022e-01,
          -7.5321e-04],
         [ 9.2305e-01, -1.5831e+00,  9.6687e-01,  3.3490e+01, -8.3052e-02,
          -3.1608e-03],
         [ 9.1957e-01, -1.0806e+00,  9.7507e-01,  3.3496e+01,  1.1131e-01,
          -2.7328e-03]],

        [[ 9.2322e-01, -4.2017e-01,  6.5789e+00,  3.1325e+01,  1.2491e+00,
          -9.6817e-03],
         [ 9.2234e-01, -3.8646e-01,  6.5784e+00,  3.1323e+01,  4.0112e-02,
           4.4220e-03],
         [ 9.2543e-01, -3.5589e-01,  6.5765e+00,  3.1324e+01, -1.8671e-01,
          -4.4127e-03],
         ...,
         [ 9.2726e-01, -1.1159e+00,  6.5904e+00,  3.1329e+01, -1.

3. Transformer/ Neural network
    1) Create a model
    2) Choose a loss function
    3) Define a dataset
    4) Set an optimizer 
    5) Run a training loop
        Calculate loss (Forward pass)
        Compute gradients (Backpropagation)
        Updating model parameters

In [ ]:
# Is scaling required?
# num_heads = width of attention (how many perspectives are considered in parallel).
# num_layers = depth of reasoning (how many times the model refines its understanding)

In [ ]:
# 3.1 TabTransformer model class, modified from Medium

class TabTransformer(nn.Module):
    def __init__(self, num_features, num_classes, dim_embedding, num_heads, num_layers):
        super(TabTransformer, self).__init__()
        self.embedding = nn.Linear(num_features, dim_embedding) # project input features -> embedding
        # transformer encoder (batch_first=True so input shape is [batch_size, timesteps, num_features/dim_embedding])
        encoder_layer = nn.TransformerEncoderLayer(d_model=dim_embedding,nhead=num_heads,dim_feedforward=dim_embedding * 4,batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.classifier = nn.Linear(dim_embedding, num_classes) # simple linear classifier

    def forward(self, x):
        # x: [batch, timesteps, features]
        x = self.embedding(x)            # -> [batch, timesteps, dim_embedding], project input to embedding
        x = self.transformer(x)          # -> [batch, timesteps, dim_embedding], passes through multiple [Attention + FFN + Norm] layers
        x = torch.mean(x, dim=1)         # -> [batch, dim_embedding], global mean pooling over timesteps 
        x = self.classifier(x)           # -> [batch, num_classes], final classification head
        return x

In [41]:
# Initialize model
model = TabTransformer(
    num_features = 6,        # 6 sensor features
    num_classes = 2,         # Binary classification
    dim_embedding = 64,      # Embedding dimension
    num_heads = 4,           # Number of attention heads
    num_layers = 4,          # Number of transformer layers
).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

print(model)

TabTransformer(
  (embedding): Linear(in_features=6, out_features=64, bias=True)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (classifier): Linear(in_features=64, out_features=2, bias=True)
)


In [35]:
# Loss function criterion and optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.004)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5)

In [28]:
# Import additional libraries
from sklearn.metrics import accuracy_score, f1_score
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
# Training loop
for epoch in range(100):
    optimizer.zero_grad()
    output = model(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')